In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 8: Heterocedasticidade (Heteroskedasticity)
Exemplo 8.7: Demanda de cigarros
             (DEMAND FOR CIGARETTES)
             
Arquivo com os dados: smoke.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.diagnostic
import numpy as np
import statsmodels.iolib.summary2 as sis

In [2]:
df = pd.read_excel('smoke.xls',
                   header=None,
                   usecols=[0, 3, 5, 6, 7, 8, 9],
                   names=['educ', 'age', 'cigs', 'restaurn', 'log_income', 'agesq', 'log_cigpric'])

In [3]:
df.head()

,educ,age,cigs,restaurn,log_income,agesq,log_cigpric
0,16.0,46,0,0,9.903487,2116,4.102743
1,16.0,40,0,0,10.308950,1600,4.058424
2,12.0,58,3,0,10.308950,3364,4.054633
3,13.5,30,0,0,9.903487,900,4.058424
4,10.0,17,0,0,9.903487,289,4.065945


### Solução sugerida

In [4]:
# Regressão pelo Método de Mínimos Quadrados Ordinários (MQO ou OLS na sigla em inglês)
# Reproduz primeira equação estimada no exemplo
# No livro consta: "A equação estimada por mínimos quadrados ponderados..." quando na realidade é MQO e não MQP

modelo_ols = smf.ols('cigs ~ log_income + log_cigpric + educ + age + agesq + restaurn', data=df)
reg_ols = modelo_ols.fit()
reg_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cigs   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     7.423
Date:                Sat, 25 Jun 2022   Prob (F-statistic):           9.50e-08
Time:                        11:56:33   Log-Likelihood:                -3236.2
No. Observations:                 807   AIC:                             6486.
Df Residuals:                     800   BIC:                             6519.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -3.6399     24.079     -0.151      0.880     -50.905      43.625
log_income      0.8803      0.728      1.210      0.227      -0.548       2.309
log_cigpric    -0.7509      5.773     -0.130      0.897     -12.084      10.582
educ           -0.5015      0.167     -3.002      0.003      -0.829      -0.174
age             0.7707      0.160      4.813      0.000       0.456       1.085
agesq          -0.0090      0.002     -5.176      0.000      -0.012      -0.006
restaurn       -2.8251      1.112     -2.541      0.011      -5.007      -0.643
==============================================================================
Omnibus:                      225.317   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              494.255
Skew:                           1.536   Prob(JB):                    4.72e-108
Kurtosis:                       5.294   Cond. No.                     1.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
'''
Teste de Breusch-Pagan da heterocedasticidade (teste BP)
H0: O Erro u é Homocedástico
H1: Violação da homocedasticidade (presença de heterocedasticidade)
'''

teste_bp = statsmodels.stats.diagnostic.het_breuschpagan(reg_ols.resid, exog_het=reg_ols.model.exog)

print(f"estatística LM: {round(teste_bp[0], 2)}")
print(f"Valor p (LM): {round(teste_bp[1], 7)}")
print(f"estatística F: {round(teste_bp[2], 2)}")
print(f"Valor p (F): {round(teste_bp[3], 3)}")

estatística LM: 32.26
Valor p (LM): 1.46e-05
estatística F: 5.55
Valor p (F): 0.0


In [ ]:
'''
Conclusão do Teste de Breusch-Pagan da heterocedasticidade

O valor-p obtido (da estat. LM) é praticamente igual a zero (note que o valor de 1.46 é multiplicado por 10 elevado a -5). 
Assim temos uma forte evidência contra H0 (ou seja, há uma forte evidência de que o termo de erro apresenta 
heterocedasticidade).
'''

In [6]:
'''
Regressão por Mínimos Quadrados Generalizados Factível (MQGF ou FGLS na sigla em inglês)
Os passos abaixo seguem o item do livro: Um procedimento MQG factível para corrigir a heterocedasticidade
Obs.: Estes passos não consistem num procedimento MQG factível geral. São válidos quando modelamos a heterocedasticidade
presumindo que:
var(u|inc) = sigma_quad * h(x)
em que: h(x) = exp(d0 + d1*log_income + d2*log_cigpric + d3*educ + d4*age + d5*agesq + d6*restaurn) 

exp(n) é um símbolo utilizado para significar "o número de euler elevado a n"
'''

# Passo 1
modelo_passo_1 = smf.ols('cigs ~ log_income + log_cigpric + educ + age + agesq + restaurn', data=df)
reg_1 = modelo_passo_1.fit()    

# Passo 2:
residuos_reg1 = reg_1.resid
log_resid_quadrado_reg_1 = np.log(np.square(residuos_reg1))

# Passo 3
modelo_passo_2 = smf.ols('log_resid_quadrado_reg_1 ~ log_income + log_cigpric + educ + age + agesq + restaurn ', data=df)
reg_2 = modelo_passo_2.fit()
valores_ajustados_reg_2 = reg_2.fittedvalues

# Passo 4
h_chapeu = np.exp(valores_ajustados_reg_2)

# Passo 5
peso = 1 / h_chapeu
modelo_mqgf = smf.wls('cigs ~ log_income + log_cigpric + educ + age + agesq + restaurn ', weights=peso, data=df)
reg_mqgf = modelo_mqgf.fit()

In [7]:
results_table = sis.summary_col(results=[reg_ols,reg_mqgf],
                            float_format='%.3f',
                            stars = False,
                            model_names=['1\n(MQO)', '2\n(MQGF)'],
                            regressor_order=['inc', 'np.square(age - 25)', 'male', 'e401k', 'Intercept'],
                            info_dict={"Núm. Obs.:": lambda x: f"{int(x.nobs)}"})

results_table.add_title('Exemplo 8.7 [Variável dependente:cigs]')
print(results_table)

Exemplo 8.7 [Variável dependente:cigs]
                  1        2    
                (MQO)    (MQGF) 
--------------------------------
Intercept      -3.640   5.635   
               (24.079) (17.803)
log_income     0.880    1.295   
               (0.728)  (0.437) 
log_cigpric    -0.751   -2.940  
               (5.773)  (4.460) 
educ           -0.501   -0.463  
               (0.167)  (0.120) 
age            0.771    0.482   
               (0.160)  (0.097) 
agesq          -0.009   -0.006  
               (0.002)  (0.001) 
restaurn       -2.825   -3.461  
               (1.112)  (0.796) 
R-squared      0.053    0.113   
R-squared Adj. 0.046    0.107   
Núm. Obs.:     807      807     
Standard errors in parentheses.


In [ ]:
'''
Nota:
Por que, no passo 5, utilizamos a sigla "mqgf" (Mínimos Quadrados Generalizados Factível) se utilizamos a função smf.wls(), 
em que wls é a sigla em inglês para Mínimos Quadrados Ponderados (Weighted Least Squares)?

Na notação do capítulo, temos que h é uma função (das variáveis explicativas) que determina a heterocedasticidade.
Se a função h é conhecida, podemos executar apenas o Passo 5 (utilizando a função h) e dizemos que rodamos a regressão pelo 
método de Mínimos Quadrados Ponderados (ou seja, utilizando o estimador de Mínimos Quadrados Ponderados).

Se a função h é desconhecida (como neste exemplo), precisamos calcular as estimativas de h para cada observação (que 
chamamos de "h_chapeu" no Passo 4).
Note que os Passos 1 a 4 são realizados para calcular "h_chapeu", que são as estimativas dos valores da função h para 
cada observação.
Neste caso em que utilizamos as estimativas "h_chapeu" por desconhecer a "verdadeira" função h, dizemos que rodamos a 
regressão pelo método de Mínimos Quadrados Generalizados Factível (ou seja, utilizando o estimador de Mínimos Quadrados 
Generalizados Factível).
'''

In [ ]:
'''
Observação sobre terminologia

Quando rodamos a regressão por MQO e por MQGF nomeamos variáveis como "modelo_ols" e "modelo_mqgf".
Nomeamos as variáveis desta forma pois o termo "modelo" é o mais usual, já os termos "ols" e "mqgf" apenas associam as 
variáveis aos métodos de estimativa utilizados.
Não queremos dizer que "estamos estimando um modelo MQO" ou um "modelo MQGF". Isto seria incorreto já que MQO e MQGF
são métodos de estimativa, não modelos.

Para detalhes, ver cap. 3 (Análise de regressão múltipla: Estimação), item 3.6 (Alguns comentários sobre a linguagem da
análise de regressão múltipla).
'''